<a href="https://colab.research.google.com/github/jonathantcallahan/guidance/blob/main/book_processing_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai

In [35]:
from openai import OpenAI
import os
import uuid
import json

client = OpenAI()

In [36]:
def store_json(chunks, targ):
    for chunk in chunks:
        if len(chunk) < 30:
            continue
        json_obj = { 
            "id" : str(uuid.uuid4()), 
            "instruction" : "You are English author and intellectual Alan Watts. Please answer the following question using your standard speech patterns but do not over-embellish.", 
            "input" : "", 
            "output" : chunk 
        }
        targ.append(json_obj)

In [5]:
def chunk_gpt(text, targ):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            { 
                "role": "system", 
                "content": "You are a document processor used to create fine-tuning data. You will receive 5000 characters worth of a book, and extract portions of text that would be coherent as the answer to a theoretical, unspecified question. Each answer can be up to 200 words. The cohesive answers within the text may directly following each other and there may be space between them that needs to be removed. The response you provide should strictly be the series of cohesive thoughts identified within the content separated by line breaks. Minor grammatical may be made as needed. "
            },
            {
                "role": "user", 
                "content": f"Extract chunks of text from this page that would be coherent as responses to an unspecified question. :\n\n{text}"
            }
        ]
    )
    store_json(response.choices[0].message.content.strip().split('\n'), targ)

In [6]:
def answer_gpt(pair):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            { 
                "role": "system", 
                "content": "You are assisting in the generation of training data for fine-tuning. You will receive a chunk of text, and will respond with a short casually phrased question to which the chunk of text you received would be an expected answer. The person who generated the answer is Alan Watt's and often he will give a response that answers a question only indirectly. The question should not exactly contain the subject matter of the answer. The question you create should be one to which the answer would be a correct indirect or metaphorical answer." 
            },
            {
                "role": "user", 
                "content": f"Return a short casually phrased question to which this text would be an appropriate. The question should not reference the core subject matter of the answer in an overt way. : {pair['output']}"
            }
        ]
    )
    pair['input'] = response.choices[0].message.content.strip()

In [7]:
filenames = []

for filename in os.listdir('books'):
    filenames.append(filename)

In [25]:
print(len(filenames))

45


In [29]:
chunk_size = 5000
text_chunks = []
processed_json = []

for i in range (2,45):
    
    with open(f'books/{filenames[i]}', 'r', encoding='utf-8', errors='replace') as file:
        content = file.read().replace('\n','')
        text_chunks += [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]
        print(f'number of chunks: {len(text_chunks)}')
        
print('books processed.')

for i in range (1, len(text_chunks) - 1):
    gpt_output = chunk_gpt(text_chunks[i], processed_json)
    print(f'answers for chunk {i} generated')

questions = 0
for entry in processed_json:
    questions += 1
    answer_gpt(entry)

print(f'questions for {questions} answers generated')

print(json.dumps(processed_json, indent=4))

number of chunks: 63
number of chunks: 127
number of chunks: 324
number of chunks: 423
number of chunks: 471
number of chunks: 568
number of chunks: 644
number of chunks: 695
number of chunks: 731
number of chunks: 788
number of chunks: 835
number of chunks: 901
number of chunks: 988
number of chunks: 1024
number of chunks: 1063
number of chunks: 1240
number of chunks: 1273
number of chunks: 1334
number of chunks: 1420
number of chunks: 1459
number of chunks: 1560
number of chunks: 1637
number of chunks: 1677
number of chunks: 1744
number of chunks: 1783
number of chunks: 1806
number of chunks: 1879
number of chunks: 1918
number of chunks: 1947
number of chunks: 2011
number of chunks: 2045
number of chunks: 2085
number of chunks: 2132
number of chunks: 2171
number of chunks: 2265
number of chunks: 2302
number of chunks: 2391
number of chunks: 2408
number of chunks: 2430
number of chunks: 2472
number of chunks: 2485
number of chunks: 2519
number of chunks: 2823
books processed.
answers 

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [30]:
with open('more_chunks', 'w') as json_file:
    json.dump(processed_json, json_file, indent=4)

In [31]:
with open('chunks_3_through_189','r') as f1:
    book_one = json.load(f1)

with open('more_chunks', 'r') as f2:
    book_two = json.load(f2)

combined_data = book_one + book_two

with open('first_batch_total', 'w') as f_combined:
    json.dump(combined_data, f_combined, indent=4)